## Supply Chain simulation demo

This notebook uses the supplychain-simulation package contained in this repository.
To install the simulator into the current kernel, run the next cell.

Jupyter will not pick up any changes made to the package automatically.  
To load your changes into the notebook:
1) run the next cell
2) restart the kernel (so the newly installed package is re-loaded)

In [1]:
! pip install ../

Processing /Users/allexveldman/PycharmProjects/chainstock-simulation
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for supplychain-simulation: filename=supplychain_simulation-0.1.0-py3-none-any.whl size=10762 sha256=efe9c35706303583087033d478d29603317c79153491e4806d39ad52e78ac044
  Stored in directory: /Users/allexveldman/Library/Caches/pip/wheels/3f/fb/a8/6cfb2eaa3f7dc9357a914634cf680bc36f7039aa7703a0cf53
Successfully built supplychain-simulation
  Attempting uninstall: supplychain-simulation
    Found existing installation: supplychain-simulation 0.1.0
    Uninstalling supplychain-simulation-0.1.0:
      Successfully uninstalled supplychain-simulation-0.1.0


In [2]:
from supplychain_simulation import SupplyChain, Node, Sales, LeadTime, Edge, Simulator
from supplychain_simulation.node import Orders, Stock
from supplychain_simulation.pipeline import Pipeline
from supplychain_simulation.receipt import Receipt
from supplychain_simulation.strategy import RSQ, Fractional

init = True

In [7]:

import logging
import sys


def setup_logging():
    logger = logging.getLogger("supplychain_simulation")
    logger.propagate = False
    logger.setLevel(logging.DEBUG)

    # create console handler and set level to debug
    # best for development or debugging
    consoleHandler = logging.StreamHandler(stream=sys.stdout)
    consoleHandler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    consoleHandler.setFormatter(formatter)

    # add handler to logger
    logger.addHandler(consoleHandler)
    global init
    init = False

if init:
    setup_logging()

### Define the supply chain.

The only manadtory field for a Node is it's `id`.

#### Sales
To simulate demand over time for a node, you can provide the `sales` field with an object that defines a `pop_sales(period: int)` method which returns the sales for that period.  
A convenience class `Sales` is provided to cover most non-dynamic use cases.
It accepts a dict where the key is the period and the value is the sales for that period.

#### Lead-time
To simulate changes in lead-time over time for a node, you can provide the `lead_time` field with an object that defines a `get_lead_time(period: int)` method which returns the lead-time for that period.  
A conveniance class `LeadTime` is provided to cover most non-dynamic use cases.
It accepts a dict where the key is the period and the value is the lead-time for that period.
in addition it also accepts a `default` that will be used for any undefined period.

#### Edges
The edges can be defined in two ways, through `Node.predecessors` and/or through `SupplyChain.edges`.
If both are present they will both be applied to the supplychain.

If an `Edge` is defined while the corresponding `Node`'s don't exist, an error will be raised.


In [8]:
supply_chain = SupplyChain(
        nodes=[
            Node(
                "A",
                data=dict(
                    order_quantity=30,
                    reorder_level=25,
                    review_time=1,
                    safety_stock=1,
                ),
                llc=0,
                sales=Sales({1: 10, 2: 10, 3: 10}),
                lead_time=LeadTime(default=1),
                stock=Stock({"A": 7, "C": 5}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=0, quantity=35)]),
            ),
            Node(
                "B",
                data=dict(
                    order_quantity=25,
                    reorder_level=40,
                    review_time=1,
                ),
                llc=0,
                sales=Sales({1: 15, 2: 15, 3: 15}),
                lead_time=LeadTime(default=2),
                pipeline=Pipeline([Receipt(sku_code="D", eta=1, quantity=75)]),
                backorders=5,
            ),
            Node(
                "C",
                data=dict(
                    order_quantity=150,
                    reorder_level=20,
                    review_time=1,
                ),
                llc=1,
                lead_time=LeadTime({1: 3, 2: 7}, default=3),
                stock=Stock({"C": 200}),
            ),
            Node(
                "D",
                data=dict(
                    order_quantity=200,
                    reorder_level=20,
                    review_time=2,
                ),
                llc=1,
                lead_time=LeadTime(default=4),
                orders=Orders({"B": 15}),
                stock=Stock({"D": 40}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=2, quantity=200)]),
            ),
        ],
        edges=[
            Edge(source="C", destination="A", number=2),
            Edge(source="D", destination="A", number=1),
            Edge(source="D", destination="B", number=3),
        ],
    )

In [9]:
sim = Simulator(
    supply_chain,
    control_strategy=RSQ(supply_chain),
    release_strategy=Fractional(),
)

In [10]:
sim.run(start_period=1, end_period=2)

2021-11-18 16:55:59,802 - supplychain_simulation.simulator - INFO - Simulate periods 1 -> 2, 2 periods
2021-11-18 16:55:59,803 - supplychain_simulation.simulator - INFO - Simulating period 1
2021-11-18 16:55:59,804 - supplychain_simulation.simulator - INFO - Receive receipts
2021-11-18 16:55:59,805 - supplychain_simulation.node - DEBUG - Node A: 1 receipts received
2021-11-18 16:55:59,806 - supplychain_simulation.node - DEBUG - 	35 D added to stock
2021-11-18 16:55:59,806 - supplychain_simulation.node - DEBUG - Node B: 0 receipts received
2021-11-18 16:55:59,807 - supplychain_simulation.node - DEBUG - Node C: 0 receipts received
2021-11-18 16:55:59,807 - supplychain_simulation.node - DEBUG - Node D: 0 receipts received
2021-11-18 16:55:59,808 - supplychain_simulation.simulator - INFO - Assemble
2021-11-18 16:55:59,808 - supplychain_simulation.node - DEBUG - Node A: Assembled 2
2021-11-18 16:55:59,809 - supplychain_simulation.node - DEBUG - Node B: Assembled 0
2021-11-18 16:55:59,810 - 